# 附录A NumPy高级应用

## A.1 ndarray对象的内部机理

In [3]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

In [ ]:
#numpy的ndarray提供了一种将同质数据块解释为多维数组对象的方式
#ndarray内部由一下内容组成：1.一个指向数据的指针
#2.数据类型或dtype,描述在数组中的固定大小值的格子
#3.一个表示数组形状(shape)的元组
#4.一个跨度元组(stride),其中整数指的是为了前进到当前维度下，一个元素需要"跨过"的字节数

In [1]:
import numpy as np
np.ones((10,5)).shape

(10, 5)

In [2]:
np.ones((10,5)).dtype

dtype('float64')

In [3]:
np.ones((10,5))

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [4]:
np.ones((10,5)).strides   #C顺序

(40, 8)

In [5]:
np.ones((3,4,5),dtype=np.float64).strides

(160, 40, 8)

In [6]:
np.ones((3,4,5),dtype=np.float64)

array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])

In [ ]:
#从（0，0，0）位置到（1，0，0）位置要160个字节bytes;
#从（0，0，0）位置到（0，1，0）位置要40个字节bytes;
#从（0，0，0）位置到（0，0，1）位置要8个字节bytes
#1bytes = 8 bits
#跨度沿一个轴上越大，沿这个轴进行计算的开销就越大
#跨度strides是构建非复制式数组视图的重要因素。跨度甚至可以是负数，这样可以使数组在内存中
#后向移动，比如在切片np[::-1]

## NumPy数据类型体系

In [ ]:
#可能需要判断数组中包含的是否是整数、浮点数、字符串或python对象
#判断dtype是否属于某个大类工作很繁琐(float就有从16到128)
#dtype都有一个超类，np.integer,np.floating

In [7]:
ints = np.ones(10,dtype=np.uint16)
floats = np.ones(10,dtype=np.float32)
np.issubdtype(ints.dtype,np.integer)

True

In [8]:
np.issubdtype(floats.dtype,np.floating)

True

In [9]:
#dtype的mro()方法可以查看其所有的父类
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [10]:
np.issubdtype(ints.dtype,np.number)

True

![numpy的dtype体系](jietu/dtype.png)

## A.2 高级数组操作

### 数组重塑

In [24]:
arr = np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [25]:
#数组从一个形状转换为另一个形状
arr.reshape((4,2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [26]:
arr   #arr源数据本未发生改变

array([0, 1, 2, 3, 4, 5, 6, 7])

In [27]:
arr.reshape((4,2),order='C')     #按C顺序（按行）

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [28]:
arr.reshape((4,2),order='F')  #按Fortran顺序（按列）

array([[0, 4],
       [1, 5],
       [2, 6],
       [3, 7]])

In [29]:
arr.reshape((4,2)).reshape((2,4))   #默认按照C顺序排列

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [30]:
arr.reshape((4,2)).reshape((2,2,2))

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [31]:
arr = np.arange(15)
arr.reshape((5,-1))       #reshape()中的参数-1表示该维度的大小由数据推断而来

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [32]:
#reshape()通常将一维数组转换为多维数组
#与reshape()相反的过程成为扁平化（flattening）或散开(raveling)
arr = np.arange(15).reshape((5,3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [33]:
arr1 = arr.ravel()   #ravel不会产生原始数据的副本

In [34]:
arr1 [:] = 0

In [35]:
arr    # arr发生了改变

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [36]:
#flatten总是返回源数据的副本
arr = np.arange(15).reshape((5,3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [37]:
arr2 = arr.flatten()

In [38]:
arr2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [39]:
arr2[:] = 0

In [40]:
arr   #arr没有发生改变

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

### C Versus Fortran Order

In [41]:
#默认下，numpy数组是按行优先顺序创建的。这意味着，每行中的数据项是被存放在相邻
#内存位置上的
#若按列优秀顺序，每列中的数据项是被存放在相邻内存位置上的

In [42]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [43]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [44]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

In [45]:
arr = np.arange(12).reshape((3,2,2))

In [46]:
arr

array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]]])

In [47]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [48]:
arr.ravel('F')

array([ 0,  4,  8,  2,  6, 10,  1,  5,  9,  3,  7, 11])

In [49]:
#C顺序：先经过更高的维度(例如，轴1会优先于轴0)
#F顺序：后经过更高的维度(例如，轴0会优先于轴1)

### Concatenating and Splitting Arrays

In [50]:
#np.concatenate可以按指定轴将一个序列连接到一起
arr1 = np.array([[1,2,3],[4,5,6]])
arr2 = np.array([[7,8,9],[10,11,12]])

In [51]:
arr1

array([[1, 2, 3],
       [4, 5, 6]])

In [52]:
arr2

array([[ 7,  8,  9],
       [10, 11, 12]])

In [53]:
np.concatenate([arr1,arr2],axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [54]:
np.concatenate([arr1,arr2],axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [56]:
#常见的连接操作vstack和hstack
np.vstack((arr1,arr2))   #竖直连接

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [57]:
np.hstack((arr1,arr2))  #水平连接

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [61]:
#split用于将一个数组沿指定轴拆分为多个数组
arr = np.random.randn(5,2)

In [62]:
arr

array([[-0.2047,  0.4789],
       [-0.5194, -0.5557],
       [ 1.9658,  1.3934],
       [ 0.0929,  0.2817],
       [ 0.769 ,  1.2464]])

In [67]:
first,second,third = np.split(arr,[1,3],axis=0)
#[1,3]表示在哪个索引出分割数组，默认axis=0

In [68]:
first

array([[-0.2047,  0.4789]])

In [69]:
second

array([[-0.5194, -0.5557],
       [ 1.9658,  1.3934]])

In [70]:
third

array([[0.0929, 0.2817],
       [0.769 , 1.2464]])

![lianjie](jietu/array_join.png)

### Stacking helpers: r_ and c_

In [71]:
#堆叠辅助类 r_ c_
arr = np.arange(6)
arr1 = arr.reshape((3,2))

In [72]:
arr2 = np.random.randn(3,2)

In [73]:
arr1

array([[0, 1],
       [2, 3],
       [4, 5]])

In [74]:
arr2

array([[ 1.0072, -1.2962],
       [ 0.275 ,  0.2289],
       [ 1.3529,  0.8864]])

In [75]:
np.r_[arr1,arr2]    #按列stacking

array([[ 0.    ,  1.    ],
       [ 2.    ,  3.    ],
       [ 4.    ,  5.    ],
       [ 1.0072, -1.2962],
       [ 0.275 ,  0.2289],
       [ 1.3529,  0.8864]])

In [76]:
np.c_[np.r_[arr1,arr2],arr]

array([[ 0.    ,  1.    ,  0.    ],
       [ 2.    ,  3.    ,  1.    ],
       [ 4.    ,  5.    ,  2.    ],
       [ 1.0072, -1.2962,  3.    ],
       [ 0.275 ,  0.2289,  4.    ],
       [ 1.3529,  0.8864,  5.    ]])

In [77]:
arr

array([0, 1, 2, 3, 4, 5])

In [78]:
arr4 = np.random.randn(6)

In [79]:
arr4

array([-2.0016, -0.3718,  1.669 , -0.4386, -0.5397,  0.477 ])

In [80]:
np.c_[arr,arr4]

array([[ 0.    , -2.0016],
       [ 1.    , -0.3718],
       [ 2.    ,  1.669 ],
       [ 3.    , -0.4386],
       [ 4.    , -0.5397],
       [ 5.    ,  0.477 ]])

In [81]:
np.r_[arr,arr4]

array([ 0.    ,  1.    ,  2.    ,  3.    ,  4.    ,  5.    , -2.0016,
       -0.3718,  1.669 , -0.4386, -0.5397,  0.477 ])

In [82]:
#r_,c_还可以将切片转换为数组
np.c_[1:6,-10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

### Repeating Elements: tile and repeat¶

In [83]:
#对数组进行重复以产生更大数组
arr = np.arange(3)
arr

array([0, 1, 2])

In [84]:
#repeat()是每个元素进行重复，并按原有顺序
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

In [85]:
#repeat()可以传入一个整数组
#第一个元素重复2次，第二个元素重复3次，第三个元素重复4次
arr.repeat([2,3,4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

In [86]:
#对于多维数组，让他们的元素沿指定轴重复,如果没有设置轴向，数组会被扁平化a
arr = np.random.randn(2,2)
arr

array([[ 3.2489, -1.0212],
       [-0.5771,  0.1241]])

In [87]:
arr.repeat(2,axis=0)

array([[ 3.2489, -1.0212],
       [ 3.2489, -1.0212],
       [-0.5771,  0.1241],
       [-0.5771,  0.1241]])

In [88]:
arr.repeat(2)

array([ 3.2489,  3.2489, -1.0212, -1.0212, -0.5771, -0.5771,  0.1241,
        0.1241])

In [89]:
arr.repeat([2,3],axis=0)

array([[ 3.2489, -1.0212],
       [ 3.2489, -1.0212],
       [-0.5771,  0.1241],
       [-0.5771,  0.1241],
       [-0.5771,  0.1241]])

In [90]:
arr.repeat([2,3],axis=1)

array([[ 3.2489,  3.2489, -1.0212, -1.0212, -1.0212],
       [-0.5771, -0.5771,  0.1241,  0.1241,  0.1241]])

In [91]:
#tite的功能是沿指定轴向堆叠数组的副本
arr

array([[ 3.2489, -1.0212],
       [-0.5771,  0.1241]])

In [92]:
np.tile(arr,2)  #第二个参数如果是标量的话，就是水平铺设

array([[ 3.2489, -1.0212,  3.2489, -1.0212],
       [-0.5771,  0.1241, -0.5771,  0.1241]])

In [94]:
np.tile(arr,(2,2))

array([[ 3.2489, -1.0212,  3.2489, -1.0212],
       [-0.5771,  0.1241, -0.5771,  0.1241],
       [ 3.2489, -1.0212,  3.2489, -1.0212],
       [-0.5771,  0.1241, -0.5771,  0.1241]])

### Fancy Indexing Equivalents: take and put

In [95]:
arr = np.arange(10) * 100

In [96]:
arr

array([  0, 100, 200, 300, 400, 500, 600, 700, 800, 900])

In [97]:
inds = [7,1,2,6]

In [98]:
#这种索引方法叫做花式索引
arr[inds]

array([700, 100, 200, 600])

In [99]:
arr.take(inds)  #找出位置是[7,1,2,6]的元素

array([700, 100, 200, 600])

In [100]:
arr.put(inds,42)  #在[7,1,2,6]的位置插入元素42,进行替换

In [101]:
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [102]:
arr.put(inds,[40,41,42,43]) #在不同的位置替换

In [103]:
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

In [106]:
inds = [2,0,2,1]
arr = np.random.randn(2,4)
arr

array([[-0.8608,  0.5601, -1.2659,  0.1198],
       [-1.0635,  0.3329, -2.3594, -0.1995]])

In [107]:
arr.take(inds,axis=1)

array([[-1.2659, -0.8608, -1.2659,  0.5601],
       [-2.3594, -1.0635, -2.3594,  0.3329]])

In [108]:
#put不接受axis参数，它只会在数组的扁平化版本上进行索引

## A.3  Broadcasting

[广播参考](https://www.cnblogs.com/jiaxin359/p/9021726.html)
[轴的理解](https://blog.csdn.net/liuweiyuxiang/article/details/80895844)

In [125]:
#广播(broadcasting)指的是不同形状的数组之间的算术运算的执行方式
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [126]:
#将标量和数组合并时会发生最简答的广播
arr * 4

array([ 0,  4,  8, 12, 16])

In [127]:
arr = np.random.randn(4,3)
arr

array([[ 1.3272, -0.9193, -1.5491],
       [ 0.0222,  0.7584, -0.6605],
       [ 0.8626, -0.01  ,  0.05  ],
       [ 0.6702,  0.853 , -0.9559]])

In [112]:
arr.mean(0)  #axis=0

array([-0.1694,  0.1863, -0.2466])

In [128]:
arr.mean(0).shape

(3,)

In [129]:
arr.shape

(4, 3)

In [114]:
#减去列平均值的方式对数组的每一列进行距平化处理
demeaned = arr - arr.mean(0)

In [115]:
demeaned

array([[-1.3726, -1.157 , -1.0605],
       [ 0.4558,  0.1917, -0.5073],
       [ 0.5007,  1.1635,  0.3164],
       [ 0.4161, -0.1981,  1.2514]])

In [116]:
demeaned.mean(0)

array([ 0., -0.,  0.])

In [130]:
#arr.mean(0).shape 为 (3,) 它是一维的，长度为3
#arr.shape 为(4,3)  它是二维的，第0维长度是4，第1维长度是3
#arr的后缘维度的长度3与arr.mean(0)的后缘维度(只有第0维)的长度3相等，于是可以广播
#缺失维是arr.mean(0)的第0维，所以沿着第0维广播

![broadcast](jietu/broadcast.png)

In [131]:
#只要遵循一定的规则，低维度的值是可以被广播到数组的任意维度的

![broadcast_pri](jietu/broadcast_principal.png)

In [132]:
arr

array([[ 1.3272, -0.9193, -1.5491],
       [ 0.0222,  0.7584, -0.6605],
       [ 0.8626, -0.01  ,  0.05  ],
       [ 0.6702,  0.853 , -0.9559]])

In [135]:
arr.shape

(4, 3)

In [133]:
row_means = arr.mean(axis=1)

In [134]:
row_means.shape

(4,)

In [136]:
#arr - row_means  不满足后缘维的要求，因此把row_means变成有维度的长度是1
#补上的维数是第1维，因此沿着第一维广播
row_means.reshape((4,1))

array([[-0.3804],
       [ 0.04  ],
       [ 0.3009],
       [ 0.1891]])

In [137]:
demeaned = arr - row_means.reshape((4,1))

In [138]:
demeaned.mean(1)

array([ 0.,  0., -0.,  0.])

In [141]:
#根据广播原则，较小数组的"广播维"的长度必须是1,因此存在一个问题：专门为了广播而添加一个长度为1的新轴
#因此numpy数组专门提供了一种通过索引机制插入轴的特殊语法
arr = np.zeros((4,4))
arr

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [142]:
arr_3d = arr[:,np.newaxis,:]

In [143]:
arr_3d

array([[[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]]])

In [144]:
arr.shape

(4, 4)

In [145]:
arr_3d.shape   #增加了一个轴，该轴的长度是1a

(4, 1, 4)

In [146]:
arr_1d = np.random.normal(size=3)

In [147]:
arr_1d

array([-0.0235, -2.3042, -0.6525])

In [150]:
arr_1d[:,np.newaxis]

array([[-0.0235],
       [-2.3042],
       [-0.6525]])

In [151]:
arr_1d[:,np.newaxis].shape

(3, 1)

In [153]:
arr_1d[np.newaxis,:].shape

(1, 3)

In [154]:
#如果我们有一个三维数组，并希望对轴2进行距平化
arr = np.random.randn(3,4,5)
arr

array([[[-1.2183, -1.3326,  1.0746,  0.7236,  0.69  ],
        [ 1.0015, -0.5031, -0.6223, -0.9212, -0.7262],
        [ 0.2229,  0.0513, -1.1577,  0.8167,  0.4336],
        [ 1.0107,  1.8249, -0.9975,  0.8506, -0.1316]],

       [[ 0.9124,  0.1882,  2.1695, -0.1149,  2.0037],
        [ 0.0296,  0.7953,  0.1181, -0.7485,  0.585 ],
        [ 0.1527, -1.5657, -0.5625, -0.0327, -0.929 ],
        [-0.4826, -0.0363,  1.0954,  0.9809, -0.5895]],

       [[ 1.5817, -0.5287,  0.457 ,  0.93  , -1.5693],
        [-1.0225, -0.4028,  0.2205, -0.1934,  0.6692],
        [-1.649 , -2.2528, -1.1668,  0.3536,  0.7021],
        [-0.2746, -0.1391,  0.1077, -0.6065, -0.4171]]])

In [155]:
depth_means = arr.mean(2)   #因为沿着轴axis=2 所有返回结果是一个shape=(3,4)
depth_means

array([[-0.0125, -0.3542,  0.0734,  0.5114],
       [ 1.0318,  0.1559, -0.5874,  0.1936],
       [ 0.1741, -0.1458, -0.8026, -0.2659]])

In [156]:
depth_means.shape

(3, 4)

In [159]:
#arr - depth_means   #会报错
demeaned = arr - depth_means[:,:,np.newaxis]

In [161]:
demeaned.mean(2)

array([[-0.,  0., -0., -0.],
       [ 0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.]])

```python
def demean_axis(arr, axis=0):
    means = arr.mean(axis)

    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]
```

### Setting Array Values by Broadcasting

In [197]:
#算术运算所遵循的广播原则同样也使用于索引机制设置数组值的操作a
arr = np.zeros((4,3))
arr

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [198]:
arr[:]  = 5

In [199]:
arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

In [200]:
col = np.array([1.28,-0.42,0.44,1.6])
arr[:] = col[:,np.newaxis]

In [201]:
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [202]:
col[:,np.newaxis]  #shape=(4,1)   而arr的shape=(4,3)  因此可以广播

array([[ 1.28],
       [-0.42],
       [ 0.44],
       [ 1.6 ]])

In [203]:
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [204]:
arr[:2] = [[-1.37],[0.509]]

In [205]:
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

In [208]:
np.array([[-1.37],[0.509]]).shape

(2, 1)

In [210]:
arr[:2].shape   #(2,1)  与 （2，3）可以广播

(2, 3)

## A.4 Advanced ufunc Usage

![numpy_ufunc](jietu/numpy_ufunc.png)

In [211]:
#reduce(x)通过连续执行原始运算的方式对值进行聚合
arr = np.arange(10)
np.add.reduce(arr)

45

In [213]:
arr.sum()

45

In [214]:
np.random.seed(123456)
arr = np.random.randn(5,5)
arr

array([[ 0.4691, -0.2829, -1.5091, -1.1356,  1.2121],
       [-0.1732,  0.1192, -1.0442, -0.8618, -2.1046],
       [-0.4949,  1.0718,  0.7216, -0.7068, -1.0396],
       [ 0.2719, -0.425 ,  0.567 ,  0.2762, -1.0874],
       [-0.6737,  0.1136, -1.4784,  0.525 ,  0.4047]])

In [216]:
arr[::2].sort(1)

In [217]:
arr  #对第1，3，5行进行了排序

array([[-1.5091, -1.1356, -0.2829,  0.4691,  1.2121],
       [-0.1732,  0.1192, -1.0442, -0.8618, -2.1046],
       [-1.0396, -0.7068, -0.4949,  0.7216,  1.0718],
       [ 0.2719, -0.425 ,  0.567 ,  0.2762, -1.0874],
       [-1.4784, -0.6737,  0.1136,  0.4047,  0.525 ]])

In [218]:
arr[:,:-1]  #丢掉最后一列

array([[-1.5091, -1.1356, -0.2829,  0.4691],
       [-0.1732,  0.1192, -1.0442, -0.8618],
       [-1.0396, -0.7068, -0.4949,  0.7216],
       [ 0.2719, -0.425 ,  0.567 ,  0.2762],
       [-1.4784, -0.6737,  0.1136,  0.4047]])

In [219]:
arr[:,1:]  #丢掉第一列

array([[-1.1356, -0.2829,  0.4691,  1.2121],
       [ 0.1192, -1.0442, -0.8618, -2.1046],
       [-0.7068, -0.4949,  0.7216,  1.0718],
       [-0.425 ,  0.567 ,  0.2762, -1.0874],
       [-0.6737,  0.1136,  0.4047,  0.525 ]])

In [220]:
arr[:,:-1] < arr[:,1:]

array([[ True,  True,  True,  True],
       [ True, False,  True, False],
       [ True,  True,  True,  True],
       [False,  True, False, False],
       [ True,  True,  True,  True]])

In [223]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:],axis=1)
#logical_and.reduce和all方法是等价的

array([ True, False,  True, False,  True])

In [227]:
(arr[:,:-1] < arr[:,1:]).all(axis=1)

array([ True, False,  True, False,  True])

In [230]:
np.add.reduce(arr[:, :-1] < arr[:, 1:],axis=1)

array([4, 2, 4, 1, 4])

In [231]:
arr = np.arange(15).reshape((3,5))
arr

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [232]:
#accumulate和reduce的关系，类似cumsum和sum的关系
np.add.accumulate(arr,axis=1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

[叉乘参考](https://blog.csdn.net/weixin_41923658/article/details/90171707)

In [248]:
#outer用于两个数组的叉乘
arr = np.arange(3).repeat([1,2,2])

In [249]:
arr

array([0, 1, 1, 2, 2])

In [250]:
np.multiply.outer(arr,np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

In [252]:
x,y = np.random.randn(3,4) , np.random.randn(5)

In [253]:
result = np.subtract.outer(x,y)

In [254]:
result

array([[[ 1.2516,  2.3539,  1.546 ,  1.8716,  0.1633],
        [-1.0404,  0.0619, -0.7461, -0.4205, -2.1287],
        [-0.3647,  0.7376, -0.0704,  0.2553, -1.453 ],
        [ 0.304 ,  1.4063,  0.5983,  0.9239, -0.7844]],

       [[-0.4833,  0.619 , -0.189 ,  0.1366, -1.5716],
        [-0.6697,  0.4326, -0.3754, -0.0498, -1.758 ],
        [ 1.5195,  2.6218,  1.8138,  2.1394,  0.4311],
        [ 1.7504,  2.8527,  2.0447,  2.3703,  0.662 ]],

       [[ 0.5656,  1.6679,  0.8599,  1.1855, -0.5228],
        [ 2.3182,  3.4205,  2.6125,  2.9381,  1.2298],
        [-0.7948,  0.3075, -0.5005, -0.1749, -1.8831],
        [ 1.0316,  2.1339,  1.3259,  1.6515, -0.0567]]])

In [255]:
result.shape   #outer输出结果的维度是两个输入数据的维度的和

(3, 4, 5)

In [256]:
#reduceate用于计算"局部约简"，起始就是一个对数据各切片进行聚合的groupby运算
arr = np.arange(10)
np.add.reduceat(arr,[0,5,8])
#返回结果是在arr[0:5]、arr[5:8] 、arr[8:]上执行add

array([10, 18, 17])

In [257]:
sum(arr[0:5])

10

In [258]:
sum(arr[5:8])

18

In [259]:
sum(arr[8:])

17

In [260]:
arr = np.multiply.outer(np.arange(4),np.arange(5))
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [261]:
np.add.reduceat(arr,[0,2,4],axis=1)

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]])

### Writing New ufuncs in Python

In [1]:
#ufunc是universal function的缩写，意思是这些函数能够作用于narray对象的每一个元素上
#有很多中方法可以让你编写自己的numpy ufuncs，最常见的是使用numpy C API

In [4]:
#numpy.frompyfunc接受一个python函数以及两个分别表示输入输出参数数量的参数
def add_elements(x,y):
    return x + y
add_them = np.frompyfunc(add_elements,2,1)

In [5]:
add_them(np.arange(8),np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

In [7]:
#用frompyfunc创建的函数总是返回python对象数组
#numpy.vectorize可以指定输出类型
add_them = np.vectorize(add_elements,otypes=[np.float64])

In [8]:
add_them(np.arange(8),np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

In [9]:
#虽然np.fromnpfunc和np.vectorize都提供了一种创建ufunc型函数的手段，但他们非常慢
#因为他们在计算每个元素时都有执行一次python函数的调用，这会比numpy自带的C的ufunc慢得多
arr = np.random.randn(10000)
%timeit add_them(arr,arr)

1.66 ms ± 33.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit np.add(arr,arr)

4.17 µs ± 94.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## A5.Structured and Record Arrays

In [12]:
#目前所讨论的ndarray都是一种同质数据容器，也就是说，在它所表示的内存块中，各个元素
#占用的字节数相同(取决于dtype)
#结构化数组是一种特殊的ndarray,其中的各个元素可以看成C语言中的结构体
dtype = [('x',np.float64),('y',np.int32)]
sarr = np.array([(1.5,6),(np.pi,-2)],dtype=dtype)

In [13]:
sarr

array([(1.5   ,  6), (3.1416, -2)], dtype=[('x', '<f8'), ('y', '<i4')])

In [14]:
sarr[0]

(1.5, 6)

In [15]:
sarr[0]['y']

6

In [16]:
sarr['x']   #返回的是该数据的视图

array([1.5   , 3.1416])

### Nested dtypes and Multidimensional Fields

In [17]:
#在定义结构化dtype时，可以再设置一个形状(可以是一个整数，也可以是一个元组)
dtype = [('x', np.int64, 3), ('y', np.int32)]
#前面3个数是np.int64
arr = np.zeros(4,dtype=dtype)

In [18]:
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

In [19]:
arr[0]

([0, 0, 0], 0)

In [20]:
arr[0]['x']

array([0, 0, 0])

In [21]:
arr['x']  #返回结果得到一个二维数组

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [23]:
#嵌套dtype，做出更复杂的结构
dtype = [('x',[('a','f8'),('b','f4')]),('y', np.int32)]

In [24]:
data = np.array([((1,2),5),((3,4),6)],dtype=dtype)

In [25]:
data

array([((1., 2.), 5), ((3., 4.), 6)],
      dtype=[('x', [('a', '<f8'), ('b', '<f4')]), ('y', '<i4')])

In [26]:
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [27]:
data['y']

array([5, 6], dtype=int32)

In [28]:
data['x']['a']

array([1., 3.])

### Why Use Structured Arrays?¶

## A6.More About Sorting

In [32]:
arr = np.random.randn(6)

In [33]:
arr

array([-0.0977, -0.6913, -2.2376, -0.1656,  0.009 , -2.6051])

In [34]:
arr.sort()

In [35]:
arr #numpy对象的sort方法是就地排序，改变源数据

array([-2.6051, -2.2376, -0.6913, -0.1656, -0.0977,  0.009 ])

In [36]:
arr = np.random.randn(3,5)
arr

array([[ 1.0853, -0.2696,  0.674 ,  0.6298, -0.586 ],
       [-0.1051,  0.369 , -0.1617,  1.2432, -1.343 ],
       [ 0.4352,  1.3676,  1.7336, -0.6718, -1.1903]])

In [37]:
arr[:,0].sort()   #视图的排序会导致源数据发生变化

In [38]:
arr

array([[-0.1051, -0.2696,  0.674 ,  0.6298, -0.586 ],
       [ 0.4352,  0.369 , -0.1617,  1.2432, -1.343 ],
       [ 1.0853,  1.3676,  1.7336, -0.6718, -1.1903]])

In [39]:
#numpy.sort方法会为数组创建一个已经排序后的副本，不是就地修改
arr = np.random.randn(5)
arr

array([ 0.0714,  0.2337,  0.3074, -1.2281,  1.7285])

In [40]:
np.sort(arr)

array([-1.2281,  0.0714,  0.2337,  0.3074,  1.7285])

In [41]:
arr   #源数据并没有发生改变

array([ 0.0714,  0.2337,  0.3074, -1.2281,  1.7285])

In [43]:
#这两个排序方法都接受一个axis参数
arr = np.random.randn(3,5)
arr

array([[ 0.6754, -0.044 , -0.6036, -0.3693, -0.9784],
       [ 1.1781, -2.8685,  0.108 ,  0.6046, -0.6212],
       [ 2.0542, -1.2113, -1.0375,  0.7093, -0.4803]])

In [44]:
arr.sort(axis=1)

In [45]:
arr

array([[-0.9784, -0.6036, -0.3693, -0.044 ,  0.6754],
       [-2.8685, -0.6212,  0.108 ,  0.6046,  1.1781],
       [-1.2113, -1.0375, -0.4803,  0.7093,  2.0542]])

In [46]:
#python的列表有一个小技巧values[::-1]可以产生一个反序的列表，ndarray也是
arr

array([[-0.9784, -0.6036, -0.3693, -0.044 ,  0.6754],
       [-2.8685, -0.6212,  0.108 ,  0.6046,  1.1781],
       [-1.2113, -1.0375, -0.4803,  0.7093,  2.0542]])

In [47]:
arr[:,::-1]

array([[ 0.6754, -0.044 , -0.3693, -0.6036, -0.9784],
       [ 1.1781,  0.6046,  0.108 , -0.6212, -2.8685],
       [ 2.0542,  0.7093, -0.4803, -1.0375, -1.2113]])

### Indirect Sorts: argsort and lexsort

In [48]:
values = np.array([5,0,1,3,2])
#对values进行排序，但返回的值是排序值的索引
indexer = values.argsort()

In [49]:
indexer   

array([1, 2, 4, 3, 0])

In [52]:
values[indexer]

array([0, 1, 2, 3, 5])

In [53]:
#根据数组的第一行对其进行排序
arr = np.random.randn(3,5)
arr

array([[-0.8566,  1.1659,  0.2808, -0.8511, -0.2201],
       [-1.1091,  0.6632, -0.66  , -0.3381,  0.635 ],
       [-0.7161,  0.0229, -0.1332, -0.1168,  0.035 ]])

In [54]:
arr[0] =  values

In [55]:
arr

array([[ 5.    ,  0.    ,  1.    ,  3.    ,  2.    ],
       [-1.1091,  0.6632, -0.66  , -0.3381,  0.635 ],
       [-0.7161,  0.0229, -0.1332, -0.1168,  0.035 ]])

In [57]:
arr[0].argsort()

array([1, 2, 4, 3, 0])

In [58]:
arr[:,arr[0].argsort()]

array([[ 0.    ,  1.    ,  2.    ,  3.    ,  5.    ],
       [ 0.6632, -0.66  ,  0.635 , -0.3381, -1.1091],
       [ 0.0229, -0.1332,  0.035 , -0.1168, -0.7161]])

In [59]:
#lexsort跟argsort差不多，只不多它可以一次性对多个键数组执行间接排序
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [60]:
sorter = np.lexsort((first_name,last_name))
#键的应用顺序是从最后一个传入算起，这里即last_name

In [61]:
sorter

array([1, 2, 3, 0, 4])

In [62]:
last_name[sorter]

array(['Arnold', 'Arnold', 'Jones', 'Jones', 'Walters'], dtype='<U7')

In [63]:
first_name[sorter]

array(['Jane', 'Steve', 'Bill', 'Bob', 'Barbara'], dtype='<U7')

In [67]:
first_name.argsort()

array([4, 3, 0, 1, 2])

In [68]:
last_name.argsort()   #这1，2是一样的;0，3是一样的，再根据first_name改变顺序

array([1, 2, 0, 3, 4])

In [70]:
list(zip(last_name[sorter],first_name[sorter]))

[('Arnold', 'Jane'),
 ('Arnold', 'Steve'),
 ('Jones', 'Bill'),
 ('Jones', 'Bob'),
 ('Walters', 'Barbara')]

### Alternative Sort Algorithms

In [71]:
values = np.array(['2:first', '2:second', '1:first', '1:second',
                   '1:third'])

In [72]:
key = np.array([2,2,1,1,1])

In [73]:
indexer = key.argsort(kind='mergesort')

In [74]:
indexer

array([2, 3, 4, 0, 1])

In [75]:
#take和put方法上面有讲
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

![sort](jietu/sort.png)

### Partially Sorting Arrays

In [76]:
#排序的目的之一可能是确定数组中最大或最小的元素
#np.partition和np.argpartition可以找到最大或最小的前k个元素 
np.random.seed(12345)
arr = np.random.randn(20)
arr

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658,  1.3934,  0.0929,
        0.2817,  0.769 ,  1.2464,  1.0072, -1.2962,  0.275 ,  0.2289,
        1.3529,  0.8864, -2.0016, -0.3718,  1.669 , -0.4386])

In [77]:
np.partition(arr,3) #结果中的头三个元素是最小的，没有特定的顺序

array([-2.0016, -1.2962, -0.5557, -0.5194, -0.3718, -0.4386, -0.2047,
        0.2817,  0.769 ,  0.4789,  1.0072,  0.0929,  0.275 ,  0.2289,
        1.3529,  0.8864,  1.3934,  1.9658,  1.669 ,  1.2464])

In [78]:
#np.argpartition返回的索引
indices = np.argpartition(arr,3)

In [79]:
indices

array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
        4, 18,  9])

In [80]:
arr.take(indices)

array([-2.0016, -1.2962, -0.5557, -0.5194, -0.3718, -0.4386, -0.2047,
        0.2817,  0.769 ,  0.4789,  1.0072,  0.0929,  0.275 ,  0.2289,
        1.3529,  0.8864,  1.3934,  1.9658,  1.669 ,  1.2464])

### numpy.searchsorted: Finding Elements in a Sorted Array

In [81]:
arr = np.array([0,1,7,12,15])
arr

array([ 0,  1,  7, 12, 15])

In [82]:
#searchsorted是一个在有序数组上执行二分查找的数组方法
#将9插入到有序的arr中，并返回其索引
arr.searchsorted(9)

3

In [83]:
arr.searchsorted([0,8,11,16]) #将每个元素独立插入的到的索引
#从结果中看到对于元素0，返回的是0，因为默认返回的是等值组的左册索引

array([0, 3, 3, 5])

In [85]:
arr = np.array([0,0,0,1,1,1,1])
arr.searchsorted([0,1])

array([0, 3])

In [86]:
arr.searchsorted([0,1],side='right')

array([3, 7])

In [87]:
#通过面元边界的数组将data数据数组拆开
data = np.floor(np.random.uniform(0, 10000, size=50))
bins = np.array([0, 100, 1000, 5000, 10000])

In [88]:
data

array([9940., 6768., 7908., 1709.,  268., 8003., 9037.,  246., 4917.,
       5262., 5963.,  519., 8950., 7282., 8183., 5002., 8101.,  959.,
       2189., 2587., 4681., 4593., 7095., 1780., 5314., 1677., 7688.,
       9281., 6094., 1501., 4896., 3773., 8486., 9110., 3838., 3154.,
       5683., 1878., 1258., 6875., 7996., 5735., 9732., 6340., 8884.,
       4954., 3516., 7142., 5039., 2256.])

In [89]:
#得到各数据点所属区间的编号，其中1表示面元[0,100)
labels = bins.searchsorted(data)

In [90]:
labels

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3,
       4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 4, 4, 3])

In [91]:
pd.Series(data).groupby(labels).mean()

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64

## A.7 Writing Fast NumPy Functions with Numba


In [92]:
#Numba是一个开源项目，它可以利用CPUs,GPUs为numpy的数据创建快速函数，
#它可以将python代码转换为机器代码

In [93]:
def mean_distance(x,y):
    """利用for循环来计算(x-y).mean()"""
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
        
    return result / count
#这个函数很慢

In [94]:
x = np.random.randn(10000000)
y = np.random.randn(10000000)

In [95]:
%timeit mean_distance(x,y)

4.79 s ± 75.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit (x-y).mean()

24.3 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [97]:
#我们可以转换这个函数为编译的Numba函数

In [99]:
import numba as nb
numba_mean_distance = nb.jit(mean_distance)

也可以写成修饰器
```python
@nb.jit
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count
```

In [101]:
%timeit numba_mean_distance(x,y)  #要比numpy自带的矢量化运算块

12.1 ms ± 547 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
#Numba的jit函数有一个选项，nonpython=True
#jit(nonpython=True)有一个简短的别名numba.njit

前面的例子也可以这样写
```python
from numba import float64, njit

@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()
```

### Creating Custom numpy.ufunc Objects with Numba

In [103]:
#numba.vectorize创建了一个编译的numpy ufunc,它与内置的ufunc很像

In [104]:
from numba import vectorize
@vectorize
def nb_add(x,y):
    return x + y

In [105]:
x = np.arange(10)

In [106]:
nb_add(x,x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [108]:
nb_add.accumulate(x)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

## A.8 Advanced Array Input and Output¶

In [110]:
#np.save和np.load可以用于读写磁盘上以二进制格式存储的数组
#内存映像(memory map)使你能处理在内存中放不下的数据集

### Memory-Mapped Files

In [111]:
#内存映像文件是一种将磁盘上的非常大的二进制数据文件当作内存中的数组进行处理的方式
#memmap对象，它允许将大文件分成小段进行读写，而不是一次性将整个数组读入内存

In [112]:
#创建一个内存映像，使用np.memmap
mmap = np.memmap('mymmap',dtype='float64', mode='w+',
                 shape=(10000, 10000))

In [113]:
mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [114]:
#对memmap切片将会返回磁盘上的数据视图
section = mmap[:5]

In [115]:
#如果将数据赋值给这些视图，数据会被先被缓存在内存中，调用flush即可写如磁盘

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
section[:] = np.random.randn(5,10000)

In [117]:
mmap.flush()

In [118]:
mmap

memmap([[ 1.3714,  0.9313,  0.6057, ..., -0.6212, -0.4678,  0.4787],
        [ 0.423 ,  0.8306,  0.6998, ...,  1.2883,  0.5886, -1.4276],
        [ 2.1601, -1.2462,  2.4447, ...,  0.8687,  0.2802,  2.1301],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]])

In [120]:
del mmap

NameError: name 'mmap' is not defined

In [121]:
#当打开一个已经存在的内存映像，仍然要指明数据类型和形状，因为磁盘上的那个文件
#只是一个二进制数据而已，没有任何元数据
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))
mmap

memmap([[ 1.3714,  0.9313,  0.6057, ..., -0.6212, -0.4678,  0.4787],
        [ 0.423 ,  0.8306,  0.6998, ...,  1.2883,  0.5886, -1.4276],
        [ 2.1601, -1.2462,  2.4447, ...,  0.8687,  0.2802,  2.1301],
        ...,
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ]])

In [122]:
%xdel mmap
!rm mymmap

### HDF5 and Other Array Storage Options

In [123]:
#PyTabels 和 h5py可以将numpy的数组数据存储为高效且可以压缩的HDF5格式

## A.9Performance Tips

In [124]:
arr_c = np.ones((1000,1000),order='C')
arr_f = np.ones((1000,1000),order='F')

In [125]:
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [126]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [127]:
arr_f.flags.f_contiguous

True

In [128]:
#对两个数组的行求和进行计算，理论上说，arr_c会比arr_f块，因为arr_c的行在内存中是连续的
%timeit arr_c.sum(1)

365 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [129]:
%timeit arr_f.sum(1)

422 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [130]:
#如果数组的内存顺序不符合你的要求，使用copy并传入'C',或‘F’
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [131]:
#在构造数组的视图时，其结果比一定是连续的
arr_c[:50].flags.contiguous

True

In [132]:
arr_c[:,:50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [133]:
%xdel arr_c
%xdel arr_f